In [ ]:
!gcloud auth login --enable-gdrive-access --update-adc

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=yUvx7awwA9bvDNu2mal1HsaoOvMDpY&prompt=consent&access_type=offline&code_challenge=rDnotczujhCQKxou9dFNXAM_OLTyY9FSAOT45hwHsAY&code_challenge_method=S256

Enter authorization code: 4/0AdQt8qj1nPysJ1ih2xVF8rd5bgOnby6FHsdFOb64CQdTHCinzh_fqhS6EjLj3Hd_e9pjLg

Application default credentials (ADC) were updated.

You are now logged in as [xinru.yang.contractor@bbva.com].
Your cu

In [1]:
!pip install pyspark
!pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 46.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=bca8b01d0409783ffb3c3e49a5690e1643c9ca954e4bbbc5fca4f882d2a0fe68
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.1 MB/s 


In [ ]:
from google.colab import auth
import gspread
from google.auth import default
from gspread.models import Worksheet
from xlsxwriter.utility import xl_col_to_name
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import row_number, monotonically_increasing_id, desc, col, when, lit, regexp_replace, from_json, max, expr, first, substring, to_date, year, month, date_format, concat_ws,count
from pyspark.sql.window import Window
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import StructType, StringType, StructField
import pyspark.sql.functions as F

In [ ]:
# Batch report

batch_report = '1yagfNNZrwn9CIhL3VeMIntzAMltmxIhRWi90HdS2NPU'
sheet_1 = 'Mes anterior'
sheet_2 = 'Nuevo Layout'

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

spark = SparkSession.builder.getOrCreate()

## 1 - Create DataFrame

In [ ]:
def create_df(report, sheet):
  """ Open worksheets and creates Dataframes"""

  worksheets = gc.open_by_key(report).worksheet(sheet)
  rows = worksheets.get_all_values()
  return spark.createDataFrame(rows, rows[0])

In [ ]:
def filter_df (dataframe,planning):
  " Filter dataframe , selecting only those that are not transferred. Drop columns without interest. We only keep ODATES and the row in which we have to map"

  filtered_df = dataframe.select(lit(0).alias("indexx"),"*")\
                        .withColumn('indexx', row_number().over(Window.orderBy(monotonically_increasing_id())))\
                        .filter(dataframe["Desplanificado"] == False)\
                        .filter(dataframe["Sin traspasar"] == True)\
                        .filter(dataframe["Solape"] == True)\
                        .filter(col("Planificación").like(planning + "%"))\
                        .drop('Job en Argos - Dataproc',
                              'Nombre Tabla',
                              'Solape',
                              'Proyecto',
                              'Ruta INPUT',
                              'Ruta OUTPUT',
                              'Último KO',
                              'Notas',
                              'Acción',
                              'Desplanificado',
                              'Sin traspasar',
                              'Actualización',
                              'Fichero Entrada',
                              'Malla',
                              'FW',
                              'Dependencia',
                              'Malla DataX',
                              'NS DataX',
                              'Transfer DataX', 
                              'Capa HDFS', 
                              'Planificación')
  filtered_df.show(20)
  return filtered_df

In [ ]:
def past_filter_df (dataframe,planning):
  " Filter dataframe , selecting only those that are not transferred. Drop columns without interest. We only keep ODATES and the row in which we have to map"

  filtered_df = dataframe.select(lit(0).alias("indexxx"),"*")\
                        .withColumn('indexxx', row_number().over(Window.orderBy(monotonically_increasing_id())))\
                        .filter(dataframe["Desplanificado"] == False)\
                        .filter(dataframe["Sin traspasar"] == True)\
                        .filter(dataframe["Solape"] == True)\
                        .filter(col("Planificación").like(planning + "%"))\
                        .drop('Job en Argos - Dataproc',
                              'Nombre Tabla',
                              'Solape',
                              'Proyecto',
                              'Ruta INPUT',
                              'Ruta OUTPUT',
                              'Último KO',
                              'Notas',
                              'Acción',
                              'Desplanificado',
                              'Sin traspasar',
                              'Actualización',
                              'Fichero Entrada',
                              'Malla',
                              'FW',
                              'Dependencia',
                              'Malla DataX',
                              'NS DataX',
                              'Transfer DataX', 
                              'Capa HDFS', 
                              'Planificación')
  filtered_df.show(20)
  return filtered_df

##2  - UnPivot the DataFrame

In [ ]:
def columns_names(dataframe):
  return dataframe.columns

In [ ]:
def unpivot_dataframe(dataframe, columns_batch, column_to_select):
  
  number_of_odates_batch = len(columns_batch[1:])

  odate_columns = columns_batch[1:]

  print("ODATE_COLUMNS",odate_columns)
  filtered_batch_df = dataframe

  for i in odate_columns:
    filtered_batch_df = filtered_batch_df.withColumnRenamed(i, str(i) + "a")
    

  filtered_batch_df.show(20,False)

  columns_batch2 = filtered_batch_df.columns
  odate_columns2 = columns_batch2[1:]
  print("DDDD-----", odate_columns2)  


  relevant_columns = odate_columns2 if odate_columns2 else filtered_batch_df.columns

  print("1-----------")

  cols_to_stack = ", ".join(['\'{c}\', {c}'.format(c=c) for c in odate_columns2]) 
  stack_expression = "stack({}, {}) as (ODATE, Status)".format(number_of_odates_batch, cols_to_stack)

  features_to_check_df = filtered_batch_df.select(*(["indexx"] + odate_columns2)).createOrReplaceTempView("features_to_check")

  sql1 = "select indexx, {} from features_to_check".format(stack_expression)
  

  unpivoted_df = spark.sql(sql1)

  print("STACKED")
  unpivoted_df = unpivoted_df.withColumn('ODATE', regexp_replace('ODATE', 'a', ''))

  return unpivoted_df


##3 - Add Date column and an increasing row_number index

In [ ]:
def add_columns(dataframe):

  # Filter our dataframe by ok/ko and add date column
  unpivoted_df = dataframe.filter((dataframe["Status"]=="OK") | (dataframe["Status"]=="KO")).withColumn("Date", to_date(col("ODATE"),"yyyyMMdd"))

  df = Window.partitionBy("indexx").orderBy("Date")

  unpivoted_df = unpivoted_df.withColumn("row_number",row_number().over(df))

  unpivoted_df.show(20)

  return unpivoted_df

##4 - Check if it is transferred

In [ ]:
def check_transfer(dataframe, n):
  checked_df = dataframe.where(dataframe.row_number <= n)

  checked_df = checked_df.groupBy("indexx").agg(count(when(col("Status")=="OK",1)).alias("num_ok")).withColumn("Traspasado", when(col("num_ok") == n,"Ha sido traspasado")\
                                                                                         #.when(col("Status") == "KO","Ha habido algún KO")\
                                                                                         .otherwise("No hay OKs suficientes"))
  print("Trapasados?")   
  checked_df.show(40)
  
  return checked_df

##5 - Select the rows that have been transferred


In [ ]:
def transferred_index(dataframe):

  transferred_df = dataframe.filter(dataframe["Traspasado"] == "Ha sido traspasado")

  transferred_df.show()

  index_list = transferred_df.select('indexx').rdd.flatMap(lambda x: x).collect()

  return index_list

In [ ]:
"""def ko_index(dataframe):

  ko_df = dataframe.filter(dataframe["Traspasado"] == "Ha habido algún KO")

  ko_df

  ko_list = ko_df.select('indexx').rdd.flatMap(lambda x: x).collect()
  ko_odate = ko_df.select('ODATE').rdd.flatMap(lambda x: x).collect()

  return ko_list,ko_odate"""

'def ko_index(dataframe):\n\n  ko_df = dataframe.filter(dataframe["Traspasado"] == "Ha habido algún KO")\n\n  ko_df\n\n  ko_list = ko_df.select(\'indexx\').rdd.flatMap(lambda x: x).collect()\n  ko_odate = ko_df.select(\'ODATE\').rdd.flatMap(lambda x: x).collect()\n\n  return ko_list,ko_odate'

## 6 - Map in worksheet

In [ ]:
def map_in_worksheet(batch,index_list, sheet):

  worksheet_batch = gc.open_by_key(batch).worksheet(sheet_2)

  for x in index_list:
    print(x)
    worksheet_batch.update("L" + str(x), False)
    worksheet_batch.update("M" + str(x), False)
    worksheet_batch.update("J" + str(x), "Ha terminado el período de solapamiento")

  #for i,x in enumerate(ko_list):
    #worksheet_batch.update("J" + str(x), "Revisar el KO en" + ko_odate[i])

In [ ]:
i = 1
def map_not_oks(batch,row_list,i,n):
  worksheet_batch = gc.open_by_key(batch).worksheet(sheet_2)
  for row in row_list:
    
    if row[2] == "KO":
      print(row[0], "i antes del reinicio ko:", i)
      print("J" + str(row[0]), "Revisar KO en " + str(row[1]))
      worksheet_batch.update("N" + str(row[0]), str(row[1]) )

      i = 0
      print(row[0], "i después del ko",  i, row[1])
      print("J" + str(row[0]), "Hay " + str(i) + " OKs")
      worksheet_batch.update("J" + str(row[0]), "Hay " + str(i) + " OKs")

    elif row[-1] == 1:
      print(row[0], "i antes del reinicio job:", i, row[1] )
      i = 1
      print(row[0], "i después de cambiar de job", i, row[1] )
      worksheet_batch.update("J" + str(row[0]), "Hay " + str(i) + " OKs")
      
    elif row[2] == "OK": 
      i = i + 1
      print(row[0], "Conteo de oks:", i, row[1])
      print("J" + str(row[0]), "Hay " + str(i) + " OKs")
      worksheet_batch.update("J" + str(row[0]), "Hay " + str(i) + " OKs")
      if i == n: 
        worksheet_batch.update("J" + str(row[0]), "Ha terminado el período de solapamiento")
        worksheet_batch.update("L" + str(row[0]), False)
        worksheet_batch.update("M" + str(row[0]), False)




## 7 - Run

In [ ]:
# Create both dataframes, this month and past month
batch_df = create_df(batch_report, sheet_2)
past_batch_df = create_df(batch_report, sheet_1)

# Filter both dfs
filtered_df = filter_df(batch_df, "D")
past_filtered_df = past_filter_df(past_batch_df, "D")

# Join dfs
joined_filtered_df = filtered_df.join(past_filtered_df,filtered_df["Job en Control-M"] ==  past_filtered_df["Job en Control-M"],"full")
joined_filtered_df = joined_filtered_df.drop("indexxx", "Job en Control-M","Job en Control-M")
joined_filtered_df.show(truncate=False)

# Unpivot dfs
columns_batchh = columns_names(joined_filtered_df)
unpivoted_df = unpivot_dataframe(joined_filtered_df, columns_batchh, "indexx")
unpivoted_df = add_columns(unpivoted_df)

checked_df = check_transfer(unpivoted_df, 10)
index_list = transferred_index(checked_df)
row_list = unpivoted_df.collect()
print(row_list[0])

# Map in worksheet
map_not_oks(batch_report,row_list, i,10)
map_in_worksheet(batch_report,index_list, sheet_2)

+------+--------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|indexx|    Job en Control-M|20220401|20220402|20220403|20220404|20220405|20220406|20220407|20220408|20220409|20220410|20220411|20220412|20220413|20220414|20220415|20220416|20220417|20220418|20220419|20220420|20220421|20220422|20220423|20220424|20220425|20220426|20220427|20220428|20220429|20220430|
+------+--------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|    12|XDTI_D_XCSF_PRICI...|        |        |        |        |        |        |        |        

In [ ]:
#batch_df = create_df(batch_report, sheet_2)
filtered_df = filter_df(batch_df, "M")
filtered_df = filtered_df.drop("Job en Control-M")
columns_batchh = columns_names(filtered_df)
unpivoted_df = unpivot_dataframe(filtered_df, columns_batchh, "indexx")
unpivoted_df = add_columns(unpivoted_df)
checked_df = check_transfer(unpivoted_df, 1)
index_list = transferred_index(checked_df)
row_list = unpivoted_df.collect()
print(row_list[0])
map_not_oks(batch_report, row_list, i, 1)
map_in_worksheet(batch_report,index_list, sheet_2)

+------+--------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|indexx|    Job en Control-M|20220401|20220402|20220403|20220404|20220405|20220406|20220407|20220408|20220409|20220410|20220411|20220412|20220413|20220414|20220415|20220416|20220417|20220418|20220419|20220420|20220421|20220422|20220423|20220424|20220425|20220426|20220427|20220428|20220429|20220430|
+------+--------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|    25|XDTI_M_XCSF_BALAN...|        |        |        |        |        |        |        |        

In [ ]:
# Create both dataframes, this month and past month
#batch_df = create_df(batch_report, sheet_2)
#past_batch_df = create_df(batch_report, sheet_1)

filtered_df = filter_df(batch_df, "S")
past_filtered_df = past_filter_df(past_batch_df, "S")

# Join dfs
joined_filtered_df = filtered_df.join(past_filtered_df,filtered_df["Job en Control-M"] ==  past_filtered_df["Job en Control-M"],"full")
joined_filtered_df = joined_filtered_df.drop("indexxx", "Job en Control-M","Job en Control-M")
joined_filtered_df.show(truncate=False)

columns_batchh = columns_names(joined_filtered_df)
unpivoted_df = unpivot_dataframe(joined_filtered_df, columns_batchh, "indexx")
unpivoted_df = add_columns(unpivoted_df)
checked_df = check_transfer(unpivoted_df, 2)
index_list = transferred_index(checked_df)
row_list = unpivoted_df.collect()
print(row_list[0])

map_not_oks(batch_report,row_list,i,2)
map_in_worksheet(batch_report,index_list, sheet_2)

+------+--------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|indexx|    Job en Control-M|20220401|20220402|20220403|20220404|20220405|20220406|20220407|20220408|20220409|20220410|20220411|20220412|20220413|20220414|20220415|20220416|20220417|20220418|20220419|20220420|20220421|20220422|20220423|20220424|20220425|20220426|20220427|20220428|20220429|20220430|
+------+--------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|    60|XSET_5000_PRICING...|        |        |        |        |        |        |        |        